In [1]:
import torch
from transformers import *

In [4]:
#tokenizer = BertTokenizer.from_pretrained("bert-large-cased")
#model = BertModel.from_pretrained("bert-base-uncased")
#lm = BertForMaskedLM.from_pretrained("bert-large-cased")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
lm = GPT2LMHeadModel.from_pretrained('gpt2-medium')

 98%|███████████████████████████████████████████████████████████▋ | 1488034816/1520013706 [1:12:30<03:13, 165433.45B/s]

OSError: Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-pytorch_model.bin' to download pretrained weights.

In [3]:
#model.eval()
lm.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [4]:
#model.to("cuda")
lm.to("cuda")

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [43]:
tokens = tokenizer.tokenize("[SEP] The boy is angry. [SEP] Someone took his favourite toy car. [SEP] He is going to [MASK]")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([token_ids, token_ids]).to("cuda")
tokens_tensor
#token_ids

tensor([[1996, 2879, 2003, 4854, 1012, 2619, 2165, 2010, 8837, 9121, 2482, 1012,
         2002, 2003, 2183, 2000,  103],
        [1996, 2879, 2003, 4854, 1012, 2619, 2165, 2010, 8837, 9121, 2482, 1012,
         2002, 2003, 2183, 2000,  103]], device='cuda:0')

In [44]:
segment_ids = [0] * len(tokens)
segments_tensor = torch.tensor([segment_ids]).to("cuda")
segments_tensor

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [45]:
with torch.no_grad():
    lm_states = lm(tokens_tensor, token_type_ids=segments_tensor)

In [46]:
len(lm_states[0][1][-1])

30522

In [47]:
predicted_index = torch.argmax(lm_states[0][0, -1]).item()
predicted_index

2022

In [48]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
predicted_token

['be']

In [49]:
torch.topk(lm_states[0][0, -1], 10)

torch.return_types.topk(
values=tensor([6.8690, 4.9379, 4.3517, 4.3496, 3.8129, 3.7198, 3.5207, 3.2630, 3.1184,
        3.0473], device='cuda:0'),
indices=tensor([2022, 3280, 2131, 1996, 2031, 2175, 4558, 2468, 2191, 2707],
       device='cuda:0'))

In [50]:
p = torch.topk(torch.softmax(lm_states[0][0][-1], 0, torch.float32), 10)
p

torch.return_types.topk(
values=tensor([0.3088, 0.0448, 0.0249, 0.0249, 0.0145, 0.0132, 0.0109, 0.0084, 0.0073,
        0.0068], device='cuda:0'),
indices=tensor([2022, 3280, 2131, 1996, 2031, 2175, 4558, 2468, 2191, 2707],
       device='cuda:0'))

In [51]:
p.values[0].item()

0.3087916374206543

In [52]:
p.indices[0].item()

2022

In [53]:
tokenizer.convert_ids_to_tokens([i.item() for i in p.indices])

['be', 'die', 'get', 'the', 'have', 'go', 'lose', 'become', 'make', 'start']

In [13]:
torch.cuda.get_device_capability(0)

(3, 5)

In [9]:
torch.backends.cudnn.enabled

True

In [53]:
tokenizer.mask_token_id

103

In [ ]:
values=tensor([9.4690, 9.1900, 7.2941, 7.1672, 7.0357], device='cuda:0'),
indices=tensor([2022, 3280, 3102, 4558, 5823], device='cuda:0'))